In [1]:
import gpflow
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from gpflow.utilities import print_summary, positive
from gpflow.config import set_default_float, default_float, set_default_summary_fmt
from gpflow.ci_utils import ci_niter

set_default_float(np.float64)
set_default_summary_fmt("notebook")


plt.style.use("ggplot")
%matplotlib inline

In [2]:
Y_all = np.loadtxt('./data/energy_olms.txt')
N = 388
n_train = 100
# I chose first 100 gbs as training data set
id_train = np.linspace(0,n_train-1, n_train).reshape(-1,1)
Y_train = Y_all[0:n_train].reshape(-1,1)
# I chose gbs from 100 to 297 as test set
id_test = np.linspace(n_train,N-1, N-n_train).reshape(-1,1)


In [3]:
class HaversineKernel(gpflow.kernels.SquaredExponential):
    """
    Isotropic RBF Kernel with Haversine distance instead of euclidean distance.
    Assumes 2 dimensional data, with columns [latitude, longitude] in degrees.
    """

    def __init__(self, lengthscales=1.0):
        super().__init__(active_dims=[0])

    def _haversine_dist(self, X, X2):
        pd = np.loadtxt('./data/pd_olms.txt')
        xx = np.unique(np.concatenate((id_test, id_train), axis= 0)).reshape(-1,1)

        len_x = len(xx)
        dist = np.zeros((len_x, len_x))
        for i in range(len_x):
            for j in range(len_x):
                dist[i,j] = pd[int(xx[i]), int(xx[j])]
        
        
#         print(X)
#         pi_on_180 = np.pi / 180

#         f = tf.expand_dims(X * pi_on_180, -2)  # ... x N x 1 x D
#         f2 = tf.expand_dims(X2 * pi_on_180, -3)  # ... x 1 x M x D

#         d = tf.sin((f - f2) / 2) ** 2
#         print(d)
#         lat1, lat2 = tf.expand_dims(X[:, 0] * pi_on_180, -1), tf.expand_dims(X2[:, 0] * pi_on_180, -2)

#         cos_prod = tf.cos(lat2) * tf.cos(lat1)

#         a = d[:, :, 0] + cos_prod * d[:, :, 1]
#         c = tf.asin(tf.sqrt(a)) * 6371 * 2
        return dist

#     @params_as_tensors
    def _scaled_square_dist(self, X, X2):
        """
        Returns (dist(X, X2ᵀ)/lengthscales)².
        """
        if X2 is None:
            X2 = X
        dist = tf.square(self._haversine_dist(X, X2) / self.lengthscales)
        return dist

In [4]:
m = gpflow.models.GPR((id_train, Y_train), kernel=HaversineKernel())

opt = gpflow.optimizers.Scipy()
opt.minimize(m.training_loss, variables=m.trainable_variables)


mean, var = m.predict_y(id_test)
# plot(m, title="trained model")

In [6]:
mean

<tf.Tensor: shape=(288, 1), dtype=float64, numpy=
array([[1.05274897],
       [1.05307145],
       [1.05339359],
       [1.05371537],
       [1.0540368 ],
       [1.05435787],
       [1.05467859],
       [1.05499895],
       [1.05531896],
       [1.05563861],
       [1.0559579 ],
       [1.05627684],
       [1.05659543],
       [1.05691365],
       [1.05723152],
       [1.05754903],
       [1.05786619],
       [1.05818298],
       [1.05849942],
       [1.0588155 ],
       [1.05913122],
       [1.05944658],
       [1.05976158],
       [1.06007622],
       [1.0603905 ],
       [1.06070442],
       [1.06101798],
       [1.06133118],
       [1.06164402],
       [1.0619565 ],
       [1.06226861],
       [1.06258036],
       [1.06289175],
       [1.06320277],
       [1.06351343],
       [1.06382373],
       [1.06413367],
       [1.06444324],
       [1.06475244],
       [1.06506128],
       [1.06536976],
       [1.06567787],
       [1.06598561],
       [1.06629299],
       [1.06660001],
     